In [ ]:
import pandas as pd
import os

# List of macroeconomic indicators to load
indicators = ['debt', 'inflation', 'reserves', 'current_account', 'fdi', 'gdp_growth']
dataframes = {}

# Load datasets for each macroeconomic indicator
for indicator in indicators:
    path = os.path.join('../data', f"{indicator}_data.csv")
    dataframes[indicator] = pd.read_csv(path)

# Known countries to identify the end of aggregates and the start of valid country data
known_countries = ['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola']

def clean_data(dataframe):
    # Loop through the 'country' column to find where aggregates end and individual country data starts
    for index, value in enumerate(dataframe['country']):
        if value in known_countries:
            countries = dataframe.iloc[index:]
            return countries.reset_index(drop = True)

    raise ValueError('No known country found in dataset. Adjust known_country list')  

In [ ]:
country_data = {}

# Clean each dataset by calling the 'clean_data' function
for indicator in indicators:
    cleaned = clean_data(dataframes[indicator]).rename(columns = {'value': indicator})
    country_data[indicator] = cleaned

In [ ]:
# Merging the datasets on common keys
merged_countries = country_data[indicators[0]]
for indicator in indicators[1:]:
    merged_countries = merged_countries.merge(country_data[indicator], on = ['country', 'iso', 'year'], how = 'inner')

In [ ]:
# Loading the regions dataframe into the notebook
regions_df = pd.read_csv('../data/country_regions.csv')

# List of countries whoes names differ between World Bank and United Nations geoscheme
mismatched_country_names = {'Bahamas': 'Bahamas, The',
    'Congo [Republic of the Congo]': 'Congo, Rep.',
    'Eswatini [Swaziland]': 'Eswatini',
    'France [French Republic]': 'France',
    'Republic of Korea [South Korea]': 'Korea, Rep.',
    'Kyrgyzstan': 'Kyrgyz Republic',
    'Micronesia (Federated States of)': 'Micronesia, Fed. Sts.',
    'Republic of Moldova': 'Moldova',
    'Slovakia': 'Slovak Republic',
    'Saint Kitts and Nevis': 'St. Kitts and Nevis',
    'Saint Lucia': 'St. Lucia',
    'Türkiye': 'Turkiye',
    'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
    'United States of America': 'United States'}

# Renaming the countries in regions_df to match those in merged_countries
regions_df['country'] = regions_df['country'].replace(mismatched_country_names)

# Merging regions_df into merged_countries
merged_countries = merged_countries.merge(regions_df, on = 'country', how = 'inner')

# Saving the fully merged and cleaned dataset
merged_countries.to_csv('../data/cleaned_data.csv', index = False)